In [2]:
import ipynb.fs.full.preprocess_eda as preprocess

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\agarw\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\agarw\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\agarw\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


The length of the training data:  7500
The length of the testing data:  4500
The length of the validation data:  3000


In [13]:
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import GlobalMaxPool1D
from tensorflow.keras.layers import Flatten

In [3]:
augment_df = preprocess.pd.read_csv(r"D:\MScDataScience\7.Data_Science_Project\SourceCode\augment_data\EDA\eda_augmented_data_size_5.csv", index_col=0)

In [7]:
train_df = preprocess.pd.concat([preprocess.train_df, augment_df], ignore_index=True)

In [9]:
train_obj = preprocess.pre_process(train_df)
val_obj = preprocess.pre_process(preprocess.val_df)
test_obj = preprocess.pre_process(preprocess.test_df)

In [10]:
x_train = train_obj.preprocess()
x_train = train_obj.lemmatise()

x_val = val_obj.preprocess()
x_val = val_obj.lemmatise()

x_test = test_obj.preprocess()
x_test = test_obj.lemmatise()

In [11]:
y_train = train_obj.encode_class()
y_val = val_obj.encode_class()
y_test = test_obj.encode_class()

In [12]:
vocab_size = 20000
sequence_length = 30

In [14]:
textvector_layer = TextVectorization(max_tokens=vocab_size, 
                                     ngrams=(1, 2, 3),
                                     output_mode="int", 
                                    #  pad_to_max_tokens=True)
                                    output_sequence_length=sequence_length)

In [15]:
textvector_layer.adapt(preprocess.pd.concat([x_train, x_val, x_test],
                                            ignore_index=True))

In [16]:
model = Sequential()
model.add(textvector_layer)
model.add(Embedding(vocab_size+2, 128))
# model.add(Conv1D(64, 
#                   kernel_size=3, 
#                   activation="relu", 
#                   padding="same"))
# model.add(Dropout(0.2))
# model.add(Conv1D(32, 
#                   kernel_size=2, 
#                   activation="relu", 
#                   padding="same"))
# model.add(Dropout(0.2))
# model.add(LSTM(100, 
#                 recurrent_dropout=0.2,
#                 activation="tanh", 
#                 return_sequences=True))
model.add(LSTM(100,
               dropout=0.2,
               recurrent_dropout=0.2,
               activation="tanh"))
# model2.add(LSTM(100, activation="tanh"))
# model2.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(150, activation="softmax"))

model.compile(optimizer=keras.optimizers.Nadam(learning_rate=0.001), 
                      loss=keras.losses.CategoricalCrossentropy(),
                      metrics=["accuracy"])


epochs = 150
batch_size = 64

history = model.fit(x_train, y_train,
                    validation_data=(x_val, y_val),
                    epochs=epochs, 
                    batch_size=batch_size,
                    callbacks=[EarlyStopping(monitor='val_accuracy', 
                                             patience=15, 
                                             min_delta=0.0001)],
                    shuffle=True)

Epoch 1/150
704/704 ━━━━━━━━━━━━━━━━━━━━ 46s 60ms/step - accuracy: 0.0140 - loss: 4.6870 - val_accuracy: 0.0503 - val_loss: 3.8024
Epoch 2/150
704/704 ━━━━━━━━━━━━━━━━━━━━ 55s 78ms/step - accuracy: 0.0615 - loss: 3.6614 - val_accuracy: 0.1053 - val_loss: 3.3519
Epoch 3/150
704/704 ━━━━━━━━━━━━━━━━━━━━ 35s 50ms/step - accuracy: 0.1311 - loss: 3.0784 - val_accuracy: 0.2737 - val_loss: 2.7238
Epoch 4/150
704/704 ━━━━━━━━━━━━━━━━━━━━ 34s 48ms/step - accuracy: 0.3123 - loss: 2.3847 - val_accuracy: 0.5100 - val_loss: 2.0026
Epoch 5/150
704/704 ━━━━━━━━━━━━━━━━━━━━ 42s 59ms/step - accuracy: 0.5683 - loss: 1.6076 - val_accuracy: 0.6647 - val_loss: 1.4415
Epoch 6/150
704/704 ━━━━━━━━━━━━━━━━━━━━ 123s 175ms/step - accuracy: 0.7645 - loss: 0.9845 - val_accuracy: 0.7333 - val_loss: 1.2188
Epoch 7/150
704/704 ━━━━━━━━━━━━━━━━━━━━ 131s 186ms/step - accuracy: 0.8603 - loss: 0.6233 - val_accuracy: 0.7603 - val_loss: 1.1083
Epoch 8/150
704/704 ━━━━━━━━━━━━━━━━━━━━ 142s 186ms/step - accuracy: 0.9050 - l

In [17]:
model2 = Sequential()
model2.add(textvector_layer)
model2.add(Embedding(vocab_size+2, 128))
model2.add(Conv1D(128, 
                  kernel_size=3, 
                  activation="relu", 
                  padding="same"))
model2.add(Dropout(0.2))
# model2.add(Conv1D(32, 
#                   kernel_size=2, 
#                   activation="relu", 
#                   padding="same"))
# model2.add(Dropout(0.2))
# model2.add(LSTM(100, 
#                 recurrent_dropout=0.2,
#                 activation="tanh", 
#                 return_sequences=True))
model2.add(LSTM(50, 
                recurrent_dropout=0.2,
                activation="tanh"))
# model2.add(LSTM(100, activation="tanh"))
# model2.add(Dropout(0.2))
model2.add(Flatten())
model2.add(Dense(150, activation="softmax"))

model2.compile(optimizer=keras.optimizers.Nadam(learning_rate=0.001), 
                      loss=keras.losses.CategoricalCrossentropy(),
                      metrics=["accuracy"])


epochs = 150
batch_size = 64

history = model2.fit(x_train, y_train,
                    validation_data=(x_val, y_val),
                    epochs=epochs, 
                    batch_size=batch_size,
                    shuffle=True,
                    callbacks=[EarlyStopping(monitor='val_accuracy', 
                                             patience=15, 
                                             min_delta=0.0001)])

Epoch 1/150
704/704 ━━━━━━━━━━━━━━━━━━━━ 44s 56ms/step - accuracy: 0.0122 - loss: 4.8203 - val_accuracy: 0.0260 - val_loss: 4.2312
Epoch 2/150
704/704 ━━━━━━━━━━━━━━━━━━━━ 36s 51ms/step - accuracy: 0.0488 - loss: 3.9975 - val_accuracy: 0.0837 - val_loss: 3.6374
Epoch 3/150
704/704 ━━━━━━━━━━━━━━━━━━━━ 41s 58ms/step - accuracy: 0.1136 - loss: 3.3119 - val_accuracy: 0.2277 - val_loss: 3.0004
Epoch 4/150
704/704 ━━━━━━━━━━━━━━━━━━━━ 35s 50ms/step - accuracy: 0.3210 - loss: 2.5128 - val_accuracy: 0.4813 - val_loss: 2.2951
Epoch 5/150
704/704 ━━━━━━━━━━━━━━━━━━━━ 35s 50ms/step - accuracy: 0.6299 - loss: 1.6158 - val_accuracy: 0.6330 - val_loss: 1.7474
Epoch 6/150
704/704 ━━━━━━━━━━━━━━━━━━━━ 35s 49ms/step - accuracy: 0.8115 - loss: 0.9846 - val_accuracy: 0.7040 - val_loss: 1.4420
Epoch 7/150
704/704 ━━━━━━━━━━━━━━━━━━━━ 35s 49ms/step - accuracy: 0.8926 - loss: 0.5954 - val_accuracy: 0.7223 - val_loss: 1.3403
Epoch 8/150
704/704 ━━━━━━━━━━━━━━━━━━━━ 35s 50ms/step - accuracy: 0.9209 - loss: 0